# Test results

With folder `data/cleaned/preds`, we can obtain the results for the position and error metrics but in an accumulated version and not as an average.

In [29]:
import glob, os, pandas as pd, numpy as np, torch, math
from tqdm import tqdm
from BronchoTrack import metrics

def compute_metrics_acum(folder):
    aape, aade, aane = [], [], []
    for pred_path in tqdm(glob.glob(folder)):
        df = pd.read_csv(pred_path, index_col=0)
        acum = df.cumsum()
        pos_preds = torch.from_numpy(acum.loc[acum.index[-1], ["shift_x", "shift_y", "shift_z"]].values)
        pos_targets = torch.from_numpy(acum.loc[acum.index[-1], ["gt_shift_x", "gt_shift_y", "gt_shift_z"]].values)
        aape.append(metrics.EuclideanDistance.euclidean(pos_preds, pos_targets).numpy())
        rot_targets = acum.loc[acum.index[-1], ["gt_Rx_dif", "gt_Ry_dif", "gt_Rz_dif"]].values
        rot_preds = acum.loc[acum.index[-1], ["Rx_dif", "Ry_dif", "Rz_dif"]].values
        # rot_targets = torch.from_numpy(np.where(rot_targets < 2*math.pi, rot_targets, rot_targets % (2*math.pi)))
        # rot_preds = torch.from_numpy(np.where(rot_preds < 2*math.pi, rot_preds, rot_preds % (2*math.pi)))
        rot_targets = torch.from_numpy(rot_targets)
        rot_preds = torch.from_numpy(rot_preds)
        aade.append(metrics.DirectionError.inverse_cos(rot_preds, rot_targets))
        aane.append(metrics.NeedleError.needle(torch.hstack([pos_preds, rot_preds]), torch.hstack([pos_targets, rot_targets]), distance=5))
    return "AAPE {} +- {}, AADE {} +- {}, AANE {} +- {}".format(np.mean(aape), np.std(aape), np.mean(aade), np.std(aade), np.mean(aane), np.std(aane))

In [42]:
compute_metrics_acum("data/cleaned/preds_15traj_MSE/*.csv"), compute_metrics_acum("data/cleaned/preds_15traj_COS/*.csv"), compute_metrics_acum("data/cleaned/preds_15traj_DE/*.csv"), compute_metrics_acum("data/cleaned/preds_15traj_QUAT/*.csv")

100%|██████████| 60/60 [00:00<00:00, 277.03it/s]


('AAPE 9.459412980342695 +- 5.1236123545985635, AADE 1.428653982441162 +- 0.6273529481583715, AANE 12.554729950272515 +- 5.149106072550288',
 'AAPE 9.476155222952693 +- 4.9495020137964865, AADE 1.6774646060124208 +- 0.6802142555063323, AANE 12.02689039269879 +- 5.46735578291376',
 'AAPE 9.638146298220851 +- 5.2773016438765765, AADE 1.5485344434354702 +- 0.5595277404864861, AANE 12.267086281305797 +- 5.290601949348858',
 'AAPE 9.534632434233725 +- 5.038242129678561, AADE 1.4740394623290076 +- 0.6164034346131992, AANE 12.527992965489972 +- 5.2862996562750695')

In [31]:
def compute_metrics(folder):
    aape, aade, aane = [], [], []
    for pred_path in tqdm(glob.glob(folder)):
        df = pd.read_csv(pred_path, index_col=0)
        for row, data in df.iterrows():
            pos_preds = torch.from_numpy(data[["shift_x", "shift_y", "shift_z"]].values)
            pos_targets = torch.from_numpy(data[["gt_shift_x", "gt_shift_y", "gt_shift_z"]].values)
            aape.append(metrics.EuclideanDistance.euclidean(pos_preds, pos_targets).numpy())
            rot_targets = data[["gt_Rx_dif", "gt_Ry_dif", "gt_Rz_dif"]].values
            rot_preds = data[["Rx_dif", "Ry_dif", "Rz_dif"]].values
            # rot_targets = torch.from_numpy(np.where(rot_targets < 2*math.pi, rot_targets, rot_targets % (2*math.pi)))
            # rot_preds = torch.from_numpy(np.where(rot_preds < 2*math.pi, rot_preds, rot_preds % (2*math.pi)))
            rot_targets = torch.from_numpy(rot_targets)
            rot_preds = torch.from_numpy(rot_preds)
            aade.append(metrics.DirectionError.inverse_cos(rot_preds, rot_targets))
            aane.append(metrics.NeedleError.needle(torch.hstack([pos_preds, rot_preds]), torch.hstack([pos_targets, rot_targets]), distance=3))
    return "AAPE {} +- {}, AADE {} +- {}, AANE {} +- {}".format(np.mean(aape), np.std(aape), np.mean(aade), np.std(aade), np.mean(aane), np.std(aane))

In [41]:
compute_metrics("data/cleaned/preds_15traj_MSE/*.csv"), compute_metrics("data/cleaned/preds_15traj_COS/*.csv"), compute_metrics("data/cleaned/preds_15traj_DE/*.csv"), compute_metrics("data/cleaned/preds_15traj_QUAT/*.csv")

100%|██████████| 60/60 [00:01<00:00, 38.05it/s]


('AAPE 0.7859296359014316 +- 0.43445301812914217, AADE 1.3504551471756292 +- 0.7871291525367192, AANE 3.6116462253046295 +- 1.726402840011931',
 'AAPE 0.7852067890908893 +- 0.42517322666672364, AADE 1.2980173913917856 +- 0.8035480129912413, AANE 3.533108055333309 +- 1.7380292609852417',
 'AAPE 0.7977475094443849 +- 0.4385815146985014, AADE 1.3602029840215764 +- 0.7983807515189764, AANE 3.662524567033723 +- 1.7101731198348715',
 'AAPE 0.7892798068412725 +- 0.4284896648112813, AADE 1.3829313858191883 +- 0.7123440449775926, AANE 3.6762834959848254 +- 1.6079433431331462')

 ## Results loss cross patient

In [36]:
from collections import defaultdict

In [50]:
def compute_errors(root="./data/cleaned/preds_round_1/", losses=["COS", "MSE", "DE", "QUAT"]):
    subfolders = {i: [os.path.join(root, folder)  for folder in os.listdir(root) if i in folder] for i in losses}
    submetrics = {i: defaultdict(list) for i in losses}
    for loss in tqdm(losses):
        for subfolder in subfolders[loss]:
            for file in os.listdir(subfolder):
                df_sample = pd.read_csv(os.path.join(subfolder, file), index_col=0)
                for row, data in df_sample.iterrows():
                    pos_preds = torch.from_numpy(data[["shift_x", "shift_y", "shift_z"]].values)
                    pos_targets = torch.from_numpy(data[["gt_shift_x", "gt_shift_y", "gt_shift_z"]].values)
                    submetrics[loss]["AAPE"].append(metrics.EuclideanDistance.euclidean(pos_preds, pos_targets).numpy())
                    rot_targets = data[["gt_Rx_dif", "gt_Ry_dif", "gt_Rz_dif"]].values
                    rot_preds = data[["Rx_dif", "Ry_dif", "Rz_dif"]].values
                    rot_targets = torch.from_numpy(rot_targets)
                    rot_preds = torch.from_numpy(rot_preds)
                    submetrics[loss]["AADE"].append(metrics.DirectionError.inverse_cos(rot_preds, rot_targets).numpy())
                    submetrics[loss]["AANE"].append(metrics.NeedleError.needle(torch.hstack([pos_preds, rot_preds]), torch.hstack([pos_targets, rot_targets]), distance=3).numpy())
        submetrics[loss]["AAPE"] = (np.mean(submetrics[loss]["AAPE"]), np.std(submetrics[loss]["AAPE"]))
        submetrics[loss]["AADE"] = (np.mean(submetrics[loss]["AADE"]), np.std(submetrics[loss]["AADE"]))
        submetrics[loss]["AANE"] = (np.mean(submetrics[loss]["AANE"]), np.std(submetrics[loss]["AANE"]))
    return submetrics
        


In [51]:
results = compute_errors()

  0%|          | 0/4 [00:00<?, ?it/s]

In [52]:
results

{'COS': defaultdict(list,
             {'AAPE': (0.7378152121278749, 0.4505294373549075),
              'AADE': (1.1975325795102736, 0.7669386701924008),
              'AANE': (3.253374431203741, 1.6608348329290958)}),
 'MSE': defaultdict(list,
             {'AAPE': (0.7555466617392615, 0.4499440452227794),
              'AADE': (1.2367228395662788, 0.7639783827566352),
              'AANE': (3.3043340700818193, 1.6125088852780254)}),
 'DE': defaultdict(list,
             {'AAPE': (0.7390488814078968, 0.4434353750507124),
              'AADE': (1.2119055731497668, 0.7702147501155583),
              'AANE': (3.2856565743090784, 1.666478388639561)}),
 'QUAT': defaultdict(list,
             {'AAPE': (0.7443752694683445, 0.44549288678195853),
              'AADE': (1.332005785650499, 0.6744398102944591),
              'AANE': (3.6061799354533117, 1.4897843823402674)})}

# Build docs for Debora

In [21]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
import torch
from BronchoTrack import metrics

In [29]:
def add_metrics(df_x):
    df_x["Direction Error"], df_x["Pos Error"], df_x["Needle Error"] = 0.0, 0.0, 0.0
    for row, data in df_x.iterrows():
            pos_preds = torch.from_numpy(data[["shift_x", "shift_y", "shift_z"]].values)
            pos_targets = torch.from_numpy(data[["gt_shift_x", "gt_shift_y", "gt_shift_z"]].values)
            df_x.loc[row, "Pos Error"] = metrics.EuclideanDistance.euclidean(pos_preds, pos_targets).numpy()
            rot_targets = data[["gt_Rx_dif", "gt_Ry_dif", "gt_Rz_dif"]].values
            rot_preds = data[["Rx_dif", "Ry_dif", "Rz_dif"]].values
            # rot_targets = torch.from_numpy(np.where(rot_targets < 2*math.pi, rot_targets, rot_targets % (2*math.pi)))
            # rot_preds = torch.from_numpy(np.where(rot_preds < 2*math.pi, rot_preds, rot_preds % (2*math.pi)))
            rot_targets = torch.from_numpy(rot_targets)
            rot_preds = torch.from_numpy(rot_preds)
            df_x.loc[row, "Direction Error"] = metrics.DirectionError.inverse_cos(rot_preds, rot_targets).numpy()
            df_x.loc[row, "Needle Error"] = metrics.NeedleError.needle(torch.hstack([pos_preds, rot_preds]), torch.hstack([pos_targets, rot_targets]), distance=3).numpy()
    return df_x


def add_info(df_x, filename, folder):
    df_x["Patient"] = filename.split("_")[1]
    df_x["Lobe"] = filename.split("_")[-2]
    df_x["Trajectory"] = filename.split("_")[-1].split(".")[0]
    df_x["Loss"] = folder.split("_")[-2]
    return df_x

In [33]:
root = "./data/cleaned/preds_round_1/"
final_csv = pd.DataFrame()
for folder in tqdm(os.listdir(root)):
    subfolder = os.path.join(root, folder)
    if os.path.isdir(subfolder):
        for file in os.listdir(subfolder):
            df_sample = pd.read_csv(os.path.join(subfolder, file), index_col=0) 
            df_sample = add_metrics(df_sample)
            df_sample = add_info(df_sample, file, folder)
            if not final_csv.empty:
                final_csv = pd.concat([final_csv, df_sample], axis=0, ignore_index=True)
            else:
                final_csv = df_sample.copy()
final_csv.to_csv(os.path.join(root, "global_result.csv"))

  0%|          | 0/20 [00:00<?, ?it/s]